In [14]:
# See https://stackoverflow.com/questions/61615818/setting-up-mlflow-on-google-colab
!pip install mlflow scikit-learn python-dotenv
!pip install pyngrok --quiet
!apt-get install nano

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nano is already the newest version (6.2-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


## TODO: set your token in .env file NGROK_AUTH_TOKEN=.... using nano
- Setting the authtoken
- Get your authtoken from https://dashboard.ngrok.com/auth

In [15]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from dotenv import load_dotenv
import os

# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Train a model
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Start an MLflow run
with mlflow.start_run():
    # Log model
    mlflow.sklearn.log_model(model, "random_forest_model")
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)

    # Print the run ID
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")

# Load the model from MLflow
logged_model = f"runs:/{run_id}/random_forest_model"
loaded_model = mlflow.sklearn.load_model(logged_model)

# Make predictions with the loaded model
loaded_pred = loaded_model.predict(X_test)
print(f"Loaded model accuracy: {accuracy_score(y_test, loaded_pred)}")

Accuracy: 1.0


2025/01/03 17:51:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: 524b11b18cfc495bbe41878245902c42
Loaded model accuracy: 1.0


In [13]:
#Start the MLflow tracking server in a separate terminal
# run "mlflow ui" in a terminal e.g. collab terminal
# Usually you access the UI at http://127.0.0.1:5000
# In this case we use nginx proxy e.g. https://88a6-34-139-58-193.ngrok-free.app

In [16]:
load_dotenv()
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://31c1-34-139-58-193.ngrok-free.app
